对stock_px 中的三个股票数据拟合ARIMA模型

ARIMA 模型对时间序列的要求是平稳型。因此，当你得到一个非平稳的时间序列时，首先要做的即是做时间序列的差分，直到得到一个平稳时间序列。如果你对时间序列做d次差分才能得到一个平稳序列，那么可以使用ARIMA(p,d,q)模型，其中d是差分次数。

In [1]:
####时序案例分析####
#参数初始化
import os 
import pandas as pd
import numpy as np

o =os.getcwd()

filename = o+"\\stock_px.csv"

close_px_all= pd.read_csv(filename,encoding="gbk", parse_dates=True, index_col=0)
close_px_all = pd.DataFrame(close_px_all,dtype=np.float64)
close_px_all.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [2]:
close_px = close_px_all[['AAPL', 'MSFT', 'XOM']]
close_px = close_px.resample('B', fill_method='ffill').ffill()
close_px.head()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: fill_method is deprecated to .resample()
the new syntax is .resample(...).ffill()
  


,AAPL,MSFT,XOM
2003-01-02,7.40,21.11,29.22
2003-01-03,7.45,21.14,29.24
2003-01-06,7.45,21.52,29.96
2003-01-07,7.43,21.93,28.95
2003-01-08,7.28,21.31,28.83


In [3]:
#差分后的结果
D_data = close_px.diff().dropna()
D_data.head()

,AAPL,MSFT,XOM
2003-01-03,0.05,0.03,0.02
2003-01-06,0.00,0.38,0.72
2003-01-07,-0.02,0.41,-1.01
2003-01-08,-0.15,-0.62,-0.12
2003-01-09,0.06,0.62,0.61


In [ ]:
AAPL = D_data[["AAPL"]]
MSFT = D_data[["MSFT"]]
XOM = D_data[["XOM"]]

In [ ]:
data.plot()
plt.show()

#自相关图
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(data).show()

#平稳性检测
from statsmodels.tsa.stattools import adfuller as ADF
print( ADF(data[u'销量']))
#返回值依次为adf、pvalue、usedlag、nobs、critical values、icbest、regresults、resstore

#差分后的结果
D_data = data.diff().dropna()
D_data.columns = [u'销量差分']
D_data.plot() #时序图
plt.show()
plot_acf(D_data).show() #自相关图

from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(D_data).show() #偏自相关图
ADF(D_data[u'销量差分'])#平稳性检测

#白噪声检验
from statsmodels.stats.diagnostic import acorr_ljungbox
acorr_ljungbox(D_data, lags=1) #返回统计量和p值

## AAPL

In [ ]:
#定阶
pmax = int(len(AAPL)/10) #一般阶数不超过length/10
qmax = int(len(AAPL)/10) #一般阶数不超过length/10

from statsmodels.tsa.arima_model import ARIMA

bic_matrix = [] #bic矩阵
for p in range(pmax+1):
  tmp = []
  for q in range(qmax+1):
    try: #存在部分报错，所以用try来跳过报错。
      tmp.append(ARIMA(AAPL, (p,1,q)).fit().bic)
    except:
      tmp.append(None)
  bic_matrix.append(tmp)

bic_matrix = pd.DataFrame(bic_matrix) #从中可以找出最小值

C:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Anaconda3\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:646: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if issubdtype(paramsdtype, float):
C:\Anaconda3\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:650: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  elif issubdtype(paramsdtype, complex):
C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'av

In [ ]:
p,q = bic_matrix.stack().idxmin() #先用stack展平，然后用idxmin找出最小值位置。
print(u'BIC最小的p值和q值为：%s、%s' %(p,q)) 
model = ARIMA(AAPL, (0,1,1)).fit() #建立ARIMA(0, 1, 1)模型
model.summary() #给出一份模型报告
model.forecast(5) #作为期5天的预测，返回预测结果、标准误差、置信区间。

In [ ]:
# bic_matrix = pd.DataFrame(bic_matrix) #从中可以找出最小值

# p,q = bic_matrix.stack().idxmin() #先用stack展平，然后用idxmin找出最小值位置。
# print(u'BIC最小的p值和q值为：%s、%s' %(p,q)) 
# model = ARIMA(close_px, (0,1,1)).fit() #建立ARIMA(0, 1, 1)模型
# model.summary() #给出一份模型报告
# model.forecast(5) #作为期5天的预测，返回预测结果、标准误差、置信区间。